In [ ]:
MUID = "benching_1_hashtagTop_9_4238446d"

In [ ]:
!pip install pillow-heif

In [ ]:
!pip install mysql-connector

In [ ]:
#the TensorFlow Model Garden repository
#!git clone https://github.com/tensorflow/models.git


In [ ]:
#!apt-get install protobuf-compiler python-lxml python-pil
#!pip install Cython pandas tf-slim lvis

In [ ]:
#cd into 'TensorFlow/models/research'
#%cd '/content/models/research'
#!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
#import os
#import sys
#os.environ['PYTHONPATH']+=":/content/models"
#sys.path.append("/content/models/research")

In [ ]:
!ls

In [ ]:
#!python /content/models/research/object_detection/packages/tf2/setup.py build
#!python /content/models/research/object_detection/packages/tf2/setup.py install

In [ ]:
##cd into 'TensorFlow/models/research/object_detection/builders/'
#%cd '/content/models/research/object_detection/builders/'
#!python model_builder_tf2_test.py
#from object_detection.utils import label_map_util
#from object_detection.utils import visualization_utils as viz_utils
#print('Done')

In [ ]:
%cd /content/

In [ ]:
#!wget "https://raw.githubusercontent.com/abundis-rmn2/graffiti_detection_OD_TF/main/labelmap.pbtxt"
#!wget "https://data.abundis.com.mx/inference_graph.tar.gz"
!wget "https://data.abundis.com.mx/config.json"
#!tar xvzf inference_graph.tar.gz
!ls

In [ ]:
import time
import argparse
import mysql.connector
import json

import requests
import io
import os
import numpy as np
import glob
import json

from six import BytesIO
from PIL import Image
from pillow_heif import register_heif_opener

register_heif_opener()
Image.LOAD_TRUNCATED_IMAGES = True
import ftplib

# For running inference on the TF-Hub module.
import tensorflow as tf

import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time
import os
# Print Tensorflow version
print(tf.__version__)

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

In [ ]:
def load_image_into_numpy_array(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

def run_inference_for_single_image(model, image):
  converted_img  = tf.image.convert_image_dtype(image, tf.float32)[tf.newaxis, ...]
  print("inside inference converted_img")
  start_time = time.time()
  output_dict = model(converted_img)
  end_time = time.time()

  #print(output_dict)
  #result = {key:value.numpy() for key,value in output_dict.items()}
  #print(result["detection_class_entities"][:5]))
  #print(result["detection_scores"][:5]))
  return output_dict

def directory_exists(dir,ftp):
    filelist = []
    ftp.retrlines('LIST',filelist.append)
    for f in filelist:
        if f.split()[-1] == dir and f.upper().startswith('D'):
            return True
    return False

def DataUpload(local_dir, target_dir):
    ftp_server.cwd('/media/exported_images_default')
    if directory_exists(target_dir, ftp_server) is False:  # (or negate, whatever you prefer for readability)
        print(target_dir)
        ftp_server.mkd(target_dir)
    ftp_server.cwd(target_dir)
    # https://stackoverflow.com/questions/67520579/uploading-a-files-in-a-folder-to-ftp-using-python-ftplib
    print("Uploading exported batch")
    toFTP = os.listdir(local_dir)
    for filename in toFTP:
        if filename not in ftp_server.nlst():
            print("Uploading: ")
            with open(os.path.join(local_dir, filename), 'rb') as file:  # Here I open the file using it's  full path
                ftp_server.storbinary(f'STOR {filename}', file)  # Here I store the file in the FTP using only it's name as I intended
            print(filename)
        else:
            print("File already exist")
    ftp_server.quit()

def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)


def download_and_resize_image(url, new_width=256, new_height=256,
                              display=False):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  response = urlopen(url)
  image_data = response.read()
  image_data = BytesIO(image_data)
  pil_image = Image.open(image_data)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  print("Image downloaded to %s." % filename)
  if display:
    display_image(pil_image)
  return filename


def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  print("inside draw_bounding_box_on_image")
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  # Reverse list and print from bottom to top.


  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin


def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.5):
  print("inside draw_boxes")
  """Overlay labeled boxes on an image with formatted scores and label names."""
  colors = list(ImageColor.colormap.values())
  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()
  print("inside draw_boxes 2")
  for i in range(min(boxes.shape[0], max_boxes)):
    print("inside draw_boxes loop")
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])

      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      print("inside draw_boxes after image_pil")
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  print("return draw_boxes")
  return image

In [ ]:
print("Load tf model")

global_time = time.time()

c = open("config.json")
config = json.load(c)

module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]

model = hub.load(module_handle).signatures['default']

In [ ]:
dir_exist = os.path.exists("./exported_images/" + MUID)
if not dir_exist:
    #os.makedirs(user_dir, 0o777)
    os.makedirs("./exported_images/" + MUID, 0o777)
    print("The dir was created")
else:
    print("The dir already exist")

dir_exist = os.path.exists("./downloaded_images/")
if not dir_exist:
    #os.makedirs(user_dir, 0o777)
    os.makedirs("./downloaded_images/", 0o777)
    print("The dir was created")
else:
    print("The dir already exist")

try:
    cnx = mysql.connector.connect(user=config["SQL"]["username"],
                                  password=config["SQL"]["password"],
                                  host=config["SQL"]["hostname"],
                                  database=config["SQL"]["database"],
                                  )
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    print("Looking for caption in MUID:", MUID)
    cursor = cnx.cursor()
    cursor.execute("SELECT * FROM data_media WHERE MUID IN ('%s') " % (MUID))
    posts = cursor.fetchall()
    print("MUID found :", len(posts))
    asset_url = ''
    img_format = ''

    for post in posts:
        inference_dict = []
        #print("Image post")
        print(post)
        print(post[6])

        #for works started
        #if post[6] == 1 and post[15] == None:
        #for barredora
        if post[6] == 1:
          print("Image post")
          print(post)
          asset_url_jpg = "https://data.abundis.com.mx/media/" + post[14] + "/" + post[1] + "_" + post[3] + ".jpg"
          asset_url_webp = "https://data.abundis.com.mx/media/" + post[14] + "/" + post[1] + "_" + post[3] + ".webp"
          asset_url_heic = "https://data.abundis.com.mx/media/" + post[14] + "/" + post[1] + "_" + post[3] + ".heic"
          print(asset_url_jpg)
          print(asset_url_webp)
          print(asset_url_heic)
          r_jpg = requests.head(asset_url_jpg)
          r_webp = requests.head(asset_url_webp)
          r_heic = requests.head(asset_url_heic)
          print(r_jpg)
          print(r_webp)
          print(r_heic)
          if r_webp.headers['Content-Type'] == 'image/webp':
            print("WEBP ",asset_url_webp)
            asset_url = asset_url_webp
            img_format= "webp"
          elif r_jpg.headers['Content-Type'] == 'image/jpeg':
            print("JPG ",asset_url_jpg)
            asset_url = asset_url_jpg
            img_format = "jpg"
          #elif r_heic.headers['Content-Type'] == 'image/heic':
          #elif r_heic=="<Response [200]>" and r_jpg=="<Response [404]>" and r_webp=="<Response [404]>":
          elif r_heic.status_code == 200:
            print("HEIC ",asset_url_heic)
            asset_url = asset_url_heic
            img_format = "heic"

          print("Image URL")
          print(asset_url)

          img_data = requests.get(asset_url).content
          if img_format == 'webp':
              with open('./downloaded_images/'+post[4]+'.webp', 'wb') as handler:
                  handler.write(img_data)
              filename = post[4]+'_exported.webp'
              try:
                image_np = load_image_into_numpy_array('./downloaded_images/' + post[4] + '.webp')
                print("inference")
                output_dict = run_inference_for_single_image(model, image_np)
                result = {key:value.numpy() for key,value in output_dict.items()}

                print("File inferences", filename)
                print("with at least 0.5 of score")
                #print(output_dict)

                for d_class, d_score in zip(output_dict['detection_class_entities'], output_dict['detection_scores']):
                    if d_score > 0.5:
                      d_class = str(d_class.numpy())
                      d_class = d_class[2:-1]
                      print('{0} with score {1}'.format(d_class, float(d_score)))
                      print(type(d_class))
                      inference_dict.append( (d_class, float(d_score)) )

                print("hay imagen?")

                image_with_boxes = draw_boxes(
                    image_np.numpy(), result["detection_boxes"],
                    result["detection_class_entities"], result["detection_scores"])
                print("hay imagen?")
                #display_image(image_with_boxes)
                im = Image.fromarray(image_with_boxes)
                im.save('./exported_images/' +MUID+ '/' + filename)

              except:
                pass



          elif img_format == 'jpg':
              with open('./downloaded_images/'+post[4]+'.jpg', 'wb') as handler:
                  handler.write(img_data)
              filename = post[4] + '_exported.jpg'
              try:
                image_np = load_image_into_numpy_array('./downloaded_images/' + post[4] + '.jpg')
                print("inference")
                output_dict = run_inference_for_single_image(model, image_np)
                result = {key:value.numpy() for key,value in output_dict.items()}

                print("File inferences", filename)
                print("with at least 0.5 of score")
                #print(output_dict)

                for d_class, d_score in zip(output_dict['detection_class_entities'], output_dict['detection_scores']):
                    if d_score > 0.5:
                      d_class = str(d_class.numpy())
                      d_class = d_class[2:-1]
                      print('{0} with score {1}'.format(d_class, float(d_score)))
                      print(type(d_class))
                      inference_dict.append( (d_class, float(d_score)) )

                print("hay imagen?")

                image_with_boxes = draw_boxes(
                    image_np.numpy(), result["detection_boxes"],
                    result["detection_class_entities"], result["detection_scores"])
                print("hay imagen?")
                #display_image(image_with_boxes)
                im = Image.fromarray(image_with_boxes)
                im.save('./exported_images/' +MUID+ '/' + filename)

              except:
                pass

          elif img_format == 'heic':
              with open('./downloaded_images/'+post[4]+'.heic', 'wb') as handler:
                  handler.write(img_data)
              filename = post[4] + '_exported.jpg'
              try:
                image_np = load_image_into_numpy_array('./downloaded_images/' + post[4] + '.heic')
                print("inference")
                output_dict = run_inference_for_single_image(model, image_np)
                result = {key:value.numpy() for key,value in output_dict.items()}

                print("File inferences", filename)
                print("with at least 0.5 of score")
                #print(output_dict)

                for d_class, d_score in zip(output_dict['detection_class_entities'], output_dict['detection_scores']):
                    if d_score > 0.5:
                      d_class = str(d_class.numpy())
                      d_class = d_class[2:-1]
                      print('{0} with score {1}'.format(d_class, float(d_score)))
                      print(type(d_class))
                      inference_dict.append( (d_class, float(d_score)) )

                print("hay imagen?")

                image_with_boxes = draw_boxes(
                    image_np.numpy(), result["detection_boxes"],
                    result["detection_class_entities"], result["detection_scores"])
                print("hay imagen?")
                #display_image(image_with_boxes)
                im = Image.fromarray(image_with_boxes)
                im.save('./exported_images/' +MUID+ '/' + filename)

              except:
                pass

          print("Inference to JSON and then SQL")
          inference_json = json.dumps(inference_dict)
          print(inference_json)
          cnx.reconnect()
          innercursor = cnx.cursor()
          sql_inference = "UPDATE data_media SET inference_world = %s WHERE id = %s"
          val = (inference_json, post[0])
          innercursor.execute(sql_inference, val)
          cnx.commit()

    print(innercursor.rowcount, "registros afectado/s")

In [ ]:
c = open("config.json")
config = json.load(c)
ftp_server = ftplib.FTP(config["FTP"]["hostname"],config["FTP"]["username"],config["FTP"]["password"])
ftp_server.encoding = "utf-8"
#ftp_server.login()

In [ ]:
DataUpload('./exported_images/' +MUID+ '/', MUID)
ftp_server.close()

In [ ]:
#!zip -r /content/portlandbench_1_hashtagTop_9_a93fe60f.zip /content/exported_images/portlandbench_1_hashtagTop_9_a93fe60f
#from google.colab import files
#files.download("/content/portlandbench_1_hashtagTop_9_a93fe60f.zip")